In [ ]:
%%capture
!pip install -qq transformers bitsandbytes peft accelerate wandb

# Optional: download best model from wandb

In [ ]:
from google.colab import userdata
import wandb
import os

os.environ["WANDB_PROJECT"] = "text-ads-generation"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints
wandb.login(key=userdata.get('WANDB_API_KEY'))

with wandb.init() as run:
  # Pass the name and version of Artifact
  my_model_name = "model-gemma-2b-9:latest"
  my_model_artifact = run.use_artifact(my_model_name)

  # Download model weights to a folder and return the path
  model_path = my_model_artifact.download()
  print('model:', model_path)

# Prediction

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

def model_summary(model):
  print('adapter:', model.active_adapters())
  summary = pd.DataFrame([(name, param.shape, param.dtype, param.requires_grad) for name, param in model.named_parameters()], columns=['name', 'shape', 'dtype', 'trainable'])
  display(summary)

# Manually set local checkpoint: uncomment below
# model_path = '/path/to/checkpoint-1234'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map='auto', torch_dtype=torch.bfloat16)
model_summary(model)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter: ['default']


,name,shape,dtype,trainable
0,model.embed_tokens.weight,"(256000, 2048)",torch.bfloat16,False
1,model.layers.0.self_attn.q_proj.base_layer.weight,"(2048, 2048)",torch.bfloat16,False
2,model.layers.0.self_attn.q_proj.lora_A.default...,"(8, 2048)",torch.bfloat16,False
3,model.layers.0.self_attn.q_proj.lora_B.default...,"(2048, 8)",torch.bfloat16,False
4,model.layers.0.self_attn.k_proj.base_layer.weight,"(256, 2048)",torch.bfloat16,False
...,...,...,...,...
411,model.layers.17.mlp.down_proj.lora_A.default.w...,"(8, 16384)",torch.bfloat16,False
412,model.layers.17.mlp.down_proj.lora_B.default.w...,"(2048, 8)",torch.bfloat16,False
413,model.layers.17.input_layernorm.weight,"(2048,)",torch.bfloat16,False
414,model.layers.17.post_attention_layernorm.weight,"(2048,)",torch.bfloat16,False


In [ ]:
def predict(model, prompt):
  with torch.no_grad():
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True, top_k=50, top_p=0.95, temperature=0.5,
                             pad_token_id= tokenizer.eos_token_id, eos_token_id= tokenizer.eos_token_id, repetition_penalty=1.2)

    # only return generated new tokens
    result = tokenizer.decode(outputs[0, inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
    return result

prompt = """### Human: Generate 10 text ads for the product with the following information.
app_name: Bauldur's Gate 3
category: Game -> RPG Game
description: Gather your party, and return to the Forgotten Realms in a tale of fellowship and betrayal, sacrifice and survival, and the lure of absolute power.
Mysterious abilities are awakening inside you, drawn from a Mind Flayer parasite planted in your brain. Resist, and turn darkness against itself. Or embrace corruption, and become ultimate evil.
From the creators of Divinity: Original Sin 2 comes a next-generation RPG, set in the world of Dungeons and Dragons

### Assistant:
"""
print('prompt:', prompt, sep='\n')
print('Model output:', predict(model, prompt), sep='\n')

prompt:
### Human: Generate 10 text ads for the product with the following information.
app_name: Bauldur's Gate 3
category: Game -> RPG Game
description: Gather your party, and return to the Forgotten Realms in a tale of fellowship and betrayal, sacrifice and survival, and the lure of absolute power.
Mysterious abilities are awakening inside you, drawn from a Mind Flayer parasite planted in your brain. Resist, and turn darkness against itself. Or embrace corruption, and become ultimate evil.
From the creators of Divinity: Original Sin 2 comes a next-generation RPG, set in the world of Dungeons and Dragons

### Assistant:

Model output:
Unleash Your Inner Monster
Mind Flayers vs Ultimate Power
Resistance versus Betrayal
Forgotten Realms Await
Party Assembled
RPG Redefined
Dueling Demons
World of Wizards
Dungeon Crawler
Final Fantasy Meets D&D
